# Day 09. Exercise 02
# Metrics

## 0. Imports

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
dayofweek = pd.read_csv('../data/dayofweek.csv')['dayofweek']
df['dayofweek'] = dayofweek
df.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [3]:
X = df.drop(columns=['dayofweek'])
y = df['dayofweek']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [5]:
def compute_metrics(y_test, pred_proba, y_pred, mode='print'):
    
    # 1. Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # 2. Precision и Recall (среднее по весам)
    precision, recall, _, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    
    # получаем количество классов
    classes = np.unique(y_test)
    # для каждого класса считаем ROC AUC против остальных
    roc_auc_list = []
    for i, class_label in enumerate(classes):
        # двоичный вектор: положительный — этот класс, остальные — отрицательные
        y_test_binary = (y_test == class_label).astype(int)
        y_score = pred_proba[:, i]
        try:
            roc = roc_auc_score(y_test_binary, y_score)
        except ValueError:
            # если есть только один класс в y_test_binary
            roc = np.nan
        roc_auc_list.append(roc)
    # взвешенное среднее по классам
    # веса — доля каждого класса в y_test
    class_counts = np.bincount(y_test)
    total = len(y_test)
    weights = class_counts / total
    roc_auc = np.sum(np.array(roc_auc_list) * weights)
    if mode == 'return':
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'roc_auc': roc_auc,
        }
    else:
        # Вывод результатов
        print(f"accuracy is {accuracy:.5f}")
        print(f"precision is {precision:.5f}")
        print(f"recall is {recall:.5f}")
        print(f"roc_auc is {roc_auc:.5f}")

In [6]:
svm = SVC(random_state=21, kernel='linear', C=10, gamma='scale', probability=True)
svm.fit(X_train, y_train)
probas = svm.predict_proba(X_test)
pred = svm.predict(X_test)
compute_metrics(y_test, probas, pred)

accuracy is 0.74556
precision is 0.74827
recall is 0.74556
roc_auc is 0.93477


## 3. Decision tree

1. The same task for decision tree

In [7]:
dtree = DecisionTreeClassifier(random_state=21, class_weight=None, criterion='entropy', max_depth=16)
dtree.fit(X_train, y_train)
probas = dtree.predict_proba(X_test)
pred = dtree.predict(X_test)
compute_metrics(y_test, probas, pred)

accuracy is 0.85207
precision is 0.85574
recall is 0.85207
roc_auc is 0.93329


## 4. Random forest

1. The same task for random forest.

In [8]:
rfor = RandomForestClassifier(random_state=21, class_weight='balanced', criterion='entropy', max_depth=16, n_estimators=100)
rfor.fit(X_train, y_train)
probas = rfor.predict_proba(X_test)
pred = rfor.predict(X_test)
compute_metrics(y_test, probas, pred)

accuracy is 0.92012
precision is 0.92304
recall is 0.92012
roc_auc is 0.98821


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [9]:
day_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}

In [10]:
classes = np.unique(y_test)
users = [col[4:] for col in X_test.columns if col.startswith('uid')]
labs = [col[8:] for col in X_test.columns if col.startswith('labname')]

errors_per_class = {}
errors_per_user = {}
errors_per_lab = {}

for cls in classes:
    indices = np.where(y_test == cls)[0]
    total_in_class = len(indices)
    pred_in_class = pred[indices]
    incorrect = np.sum(pred_in_class != cls)
    error_percent = (incorrect / total_in_class) * 100 if total_in_class > 0 else 0
    errors_per_class[cls] = error_percent

for cls in classes:
    # Создаём маску для объектов этого класса
    class_mask = (y_test == cls)
    total_in_class = np.sum(class_mask)

    if total_in_class == 0:
        error_percent = 0
    else:
        incorrect = np.sum((pred != cls) & class_mask)
        error_percent = (incorrect / total_in_class) * 100

    errors_per_class[cls] = error_percent

# Анализ по пользователям
for user in users:
    col_name = 'uid_' + user
    user_mask = X_test[col_name]
    # Булева маска - где пользователь активен
    boolean_mask = user_mask.astype(bool)
    total = np.sum(boolean_mask)
    if total > 0:
        incorrect = np.sum((pred != y_test) & boolean_mask)
        errors_per_user[user] = (incorrect / total) * 100
    else:
        errors_per_user[user] = 0

# Анализ по лабораториям
for lab in labs:
    col_name = 'labname_' + lab
    lab_mask = X_test[col_name]
    boolean_mask = lab_mask.astype(bool)
    total = np.sum(boolean_mask)
    if total > 0:
        incorrect = np.sum((pred != y_test) & boolean_mask)
        errors_per_lab[lab] = (incorrect / total) * 100
    else:
        errors_per_lab[lab] = 0

# Находим максимум
most_error_pro_class = max(errors_per_class, key=errors_per_class.get)
max_error_percentage = errors_per_class[most_error_pro_class]

most_error_pro_user = max(errors_per_user, key=errors_per_user.get)
max_error_user = errors_per_user[most_error_pro_user]

most_error_pro_lab = max(errors_per_lab, key=errors_per_lab.get)
max_error_lab = errors_per_lab[most_error_pro_lab]


print(f"Most mistakes in class: {day_names[most_error_pro_class]} ({max_error_percentage:.2f}%)")
print(f"User with most errors: {most_error_pro_user} ({max_error_user:.2f}%)")
print(f"Lab with most errors: {most_error_pro_lab} ({max_error_lab:.2f}%)")

Most mistakes in class: Monday (22.22%)
User with most errors: user_6 (50.00%)
Lab with most errors: lab03 (100.00%)


In [11]:
joblib.dump(rfor, 'random_forest.pkl')

['random_forest.pkl']

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [12]:
def evaluate_models(models, params, X_train, y_train, X_test, y_test):
    results = {}
    
    for model, param in zip(models, params):
        # Инициализация модели с параметрами
        model_instance = model(**param)
        
        # Обучение модели
        model_instance.fit(X_train, y_train)
        
        # Предсказания
        y_pred = model_instance.predict(X_test)
        y_proba = model_instance.predict_proba(X_test)
        # Вычисление метрик
        d = compute_metrics(y_test, y_proba, y_pred, mode='return')
        
        # сохраняем результаты
        model_name = model.__name__
        results[model_name] = d
    
    return results

In [13]:
models = [SVC, DecisionTreeClassifier, RandomForestClassifier]
params = [
    {'random_state': 21, 'kernel': 'linear', 'C': 10, 'gamma': 'scale', 'probability': True},
    {'random_state': 21, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 16},
    {'random_state': 21, "class_weight": 'balanced', "criterion": 'entropy', "max_depth": 16, 'n_estimators': 100},
    
]

results = evaluate_models(models, params, X_train, y_train, X_test, y_test)
# print(results)
for key, value in results.items():
    print(f'{key}:')
    for metric_key, metric_value in value.items():
        print(f'    {metric_key}: {metric_value:.5f}')
    print()

SVC:
    accuracy: 0.74556
    precision: 0.74827
    recall: 0.74556
    roc_auc: 0.93477

DecisionTreeClassifier:
    accuracy: 0.85207
    precision: 0.85574
    recall: 0.85207
    roc_auc: 0.93329

RandomForestClassifier:
    accuracy: 0.92012
    precision: 0.92304
    recall: 0.92012
    roc_auc: 0.98821

